In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel


In [14]:
data = pd.read_csv("../data/df_pooled_ranking_clean.csv").drop(columns=["nome"])

data.head()


,ano,anos_pm,bolsista_encoded,cg,diff_fase,iaa,ian,ida,idade,ieg,inde,ipp,ips,ipv,na_fase,ponto_virada_encoded,sexo_encoded
0,2022,0.0,0,245.0,-2.0,10.00002,5.0,8.816667,11.0,7.581705,7.675509,5.6250,7.500,7.916665,0,0,1.0
1,2020,2.0,0,451.0,-1.0,8.00002,5.0,0.000000,10.0,4.100000,5.076252,7.1875,6.875,7.750000,0,0,0.0
2,2022,0.0,0,451.0,-1.0,10.00002,5.0,5.083333,12.0,8.071429,7.117065,6.2500,7.500,8.055553,0,0,0.0
3,2020,1.0,0,415.0,0.0,7.50002,10.0,7.500000,10.0,8.000000,8.077085,8.4375,7.500,8.166665,1,0,0.0
4,2021,2.0,0,415.0,0.0,7.40000,10.0,5.400000,11.0,7.500000,7.399000,8.5000,7.500,7.400000,1,0,0.0


In [15]:
data = pd.get_dummies(data, columns=["ano"], drop_first=True, dtype=int)

data.head()


,anos_pm,bolsista_encoded,cg,diff_fase,iaa,ian,ida,idade,ieg,inde,ipp,ips,ipv,na_fase,ponto_virada_encoded,sexo_encoded,ano_2021,ano_2022
0,0.0,0,245.0,-2.0,10.00002,5.0,8.816667,11.0,7.581705,7.675509,5.6250,7.500,7.916665,0,0,1.0,0,1
1,2.0,0,451.0,-1.0,8.00002,5.0,0.000000,10.0,4.100000,5.076252,7.1875,6.875,7.750000,0,0,0.0,0,0
2,0.0,0,451.0,-1.0,10.00002,5.0,5.083333,12.0,8.071429,7.117065,6.2500,7.500,8.055553,0,0,0.0,0,1
3,1.0,0,415.0,0.0,7.50002,10.0,7.500000,10.0,8.000000,8.077085,8.4375,7.500,8.166665,1,0,0.0,0,0
4,2.0,0,415.0,0.0,7.40000,10.0,5.400000,11.0,7.500000,7.399000,8.5000,7.500,7.400000,1,0,0.0,1,0


In [16]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1646 entries, 0 to 1645
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   anos_pm               1646 non-null   float64
 1   bolsista_encoded      1646 non-null   int64  
 2   cg                    1646 non-null   float64
 3   diff_fase             1646 non-null   float64
 4   iaa                   1646 non-null   float64
 5   ian                   1646 non-null   float64
 6   ida                   1646 non-null   float64
 7   idade                 1646 non-null   float64
 8   ieg                   1646 non-null   float64
 9   inde                  1646 non-null   float64
 10  ipp                   1646 non-null   float64
 11  ips                   1646 non-null   float64
 12  ipv                   1646 non-null   float64
 13  na_fase               1646 non-null   int64  
 14  ponto_virada_encoded  1646 non-null   int64  
 15  sexo_encoded         

In [19]:
# Compute the correlation matrix to check for multicollinearity
correlation_matrix = data.corr()

correlation_matrix.style.background_gradient(cmap="coolwarm")


,anos_pm,bolsista_encoded,cg,diff_fase,iaa,ian,ida,idade,ieg,inde,ipp,ips,ipv,na_fase,ponto_virada_encoded,sexo_encoded,ano_2021,ano_2022
anos_pm,1.000000,0.511746,0.075968,0.009785,-0.095882,-0.009634,-0.064799,0.388553,-0.130786,-0.082663,0.228478,-0.201797,0.003945,-0.041668,0.061084,0.002636,0.079355,-0.063842
bolsista_encoded,0.511746,1.000000,-0.249112,0.234790,-0.009684,0.223288,0.180252,0.254114,0.118261,0.238295,0.294217,-0.101830,0.247107,0.150474,0.256865,-0.003454,0.081869,-0.108149
cg,0.075968,-0.249112,1.000000,-0.354565,-0.323269,-0.351756,-0.598294,0.322529,-0.575793,-0.752786,-0.216040,-0.247108,-0.599130,-0.300789,-0.434532,-0.001174,0.010137,-0.025361
diff_fase,0.009785,0.234790,-0.354565,1.000000,0.058336,0.854130,0.135258,-0.336294,0.155653,0.382036,0.221482,0.083749,0.184852,0.705142,0.133543,-0.007697,0.071254,-0.180757
iaa,-0.095882,-0.009684,-0.323269,0.058336,1.000000,0.062601,0.239410,-0.230479,0.309920,0.461080,0.005567,0.140351,0.267823,0.050215,0.136843,0.009544,-0.014711,-0.055035
ian,-0.009634,0.223288,-0.351756,0.854130,0.062601,1.000000,0.123170,-0.311121,0.135779,0.380627,0.160474,0.038998,0.138295,0.938138,0.107254,-0.005651,0.047169,-0.154339
ida,-0.064799,0.180252,-0.598294,0.135258,0.239410,0.123170,1.000000,-0.246057,0.571757,0.802749,0.125314,0.129288,0.519352,0.100663,0.288878,-0.005796,-0.134558,-0.056090
idade,0.388553,0.254114,0.322529,-0.336294,-0.230479,-0.311121,-0.246057,1.000000,-0.316850,-0.330440,0.261426,-0.124257,-0.156395,-0.294524,-0.028670,-0.019123,-0.007964,0.114700
ieg,-0.130786,0.118261,-0.575793,0.155653,0.309920,0.135779,0.571757,-0.316850,1.000000,0.795627,0.072253,0.099689,0.555251,0.110449,0.286676,-0.039263,-0.203363,0.049032
inde,-0.082663,0.238295,-0.752786,0.382036,0.461080,0.380627,0.802749,-0.330440,0.795627,1.000000,0.352338,0.296969,0.774785,0.329610,0.439161,-0.012108,-0.051939,-0.133621


In [21]:
# Importing necessary libraries for ordinal logistic regression
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

# Selecting relevant features based on the previous analysis
selected_features = [
    "idade",
    "inde",
    "ieg",
    "ipv",
    "ida",
    "diff_fase",
    "sexo_encoded",
    "bolsista_encoded",
]

# Preparing the data
X = data[selected_features]
y = data["cg"]

# Ordinal Logistic Regression model (Proportional Odds Model)
model = OrderedModel(y, X, distr="logit")
result = model.fit(method="bfgs")

# Display the summary of the model
result.summary()


KeyboardInterrupt: 

In [25]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Preparing the dataset
X = data.drop(columns=["cg"])
y = data["cg"]

# Splitting the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Initializing the CatBoost model for ordinal classification
catboost_model = CatBoostClassifier(
    loss_function="MultiClass",
    eval_metric="Accuracy",
    verbose=0,
    task_type="GPU",  # Enable GPU
    devices="0",  # Specify GPU device (optional, default is the first GPU)
    random_seed=42,
)

# Fitting the model on the training data
catboost_model.fit(X_train, y_train)

# Making predictions on the test data
y_pred = catboost_model.predict(X_test)

# Evaluating the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

# Displaying the first 10 predictions and their corresponding true values
print("Predictions: ", y_pred[:10])
print("True values: ", y_test[:10].values)


KeyboardInterrupt: 